In [2]:
import pandas as pd
from pathlib import Path

from data_tools.df_processing import char_combine_iter 
from data_tools.wiki import node_query_pipeline

# New line recommended by notebook
from tqdm.autonotebook import tqdm 

/Users/sulhasan/opt/anaconda3/lib/python3.8/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
